In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

In [ ]:
## Some library
from address_convertor import pubkey_to_address, get_pubkey
from time_manager import get_time

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 60*5

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

In [ ]:
def get_data(theight):
    blks = list()
    txes = list()
    addrs = list()
    block_hash = rpc_connection.getblockhash(theight)
    block = rpc_connection.getblock(block_hash, 2)
    blks.append(block_hash)
    for tx in block['tx']:
        txes.append(tx['txid']) # IMPORTRANT! We should to use txid not hash
        n = -1
        for vout in tx['vout']:
            n = n + 1
            if vout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                'witness_v0_keyhash', 'witness_v0_scripthash',
                                                'witness_unknown', 'multisig'):
                for addr in vout['scriptPubKey']['addresses']:
                    addrs.append(addr)
            elif vout['scriptPubKey']['type'] in ('pubkey', 'nonstandard', 'nulldata'):
                try:
                    for addr in [pubkey_to_address(get_pubkey(vout['scriptPubKey']['hex']))]:
                        addrs.append(addr)
                except UnboundLocalError:
                    for addr in [f'{tx["txid"]}{n}']:
                        addrs.append(addr)
            else:
                raise SystemExit(f'BUG!! {theight}\n{tx}\n{vout}')
    return blks, txes, addrs

In [ ]:
val_data = './val_data' # default
val_root = os.path.abspath(os.path.expanduser(val_data))
os.makedirs(val_root, exist_ok=True)

In [ ]:
import time
import json

buffer = 1000
start_height = 0
end_height = (best_block['height']//buffer-1)*buffer

rpc_connection = get_rpc()
stime = time.time()
for height in range(start_height, end_height+1):
    result = get_data(height)
    data = {'blkhash': result[0],
            'txid': result[1],
            'addr': result[2]}
    path_dir = os.path.join(val_root, f'{height//buffer*buffer}')
    os.makedirs(path_dir, exist_ok=True)
    path = os.path.join(path_dir, f'{height}.json')
    try:
        with open(path, 'w') as f:
            json.dump(data, f)
    except KeyboardInterrupt:
        with open(path, 'w') as f:
            json.dump(data, f)
        break
    if height%buffer == 0:
        etime = time.time()
        print(f'{height} Done: {etime-stime}')
        stime = time.time()